# **Create a poker hand game and probability simulator**
## - Set up Classes for Cards, Decks and Players.  
## - Create functions and methods to classify any poker hand and determine the best one given a Player's hand and the board (for Hold'em variants). 
## - Create other functions to run simulations for individual players to see the probability of their final hand or the probability of any player in a group winning at the end.

### *Import required packages*

In [139]:
import random
import uuid
from functools import total_ordering
from itertools import product, combinations, accumulate
import collections
import functools
import operator
import getpass
import time

### *Set up global lists and dictionaries*

In [140]:
CARD_VALS = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"]
SUITS = ["Spades", "Hearts", "Diamonds", "Clubs"]
HAND_ORDER_old = {'High card':0, 'One pair':1, 'Two pair':2, 'Three of a kind':3,
             'Straight':4, 'Flush':5, 'Full house':6, 'Four of a kind':7, 'Straight flush':8}
HAND_ORDER = {v:k for k,v in HAND_ORDER_old.items()}
PLAYER_DICT = {} #A dictionary of player.id:player

## The **Card** class is set up with a VALUE (from CARD_VALS) and a SUIT

In [141]:
@total_ordering
class Card:

    def __init__(self, val, suit):
        self.suit = suit
        self.val = val
        self.id = uuid.uuid4()
        self.index = CARD_VALS.index(val)

    def __str__(self):
        return "{} of {}".format(self.val, self.suit)

    def __repr__(self):
        return "Card({}, {})".format(self.val, self.suit)

    def __eq__(self, other):
        return self.val == other.val

    def __le__(self, other):
        return self.index <= other.index
    
    def __lt__(self, other):
        return self.index < other.index

## The **Deck** class creates a deck of 52 Cards and establishes a **board**.
### It has methods for **shuffling, drawing** cards, **adding** and **removing** cards and **adding to the board**

In [142]:
class Deck:

    def __init__(self):
        self.cards = []
        self.board = []
        self.setup()

    def __len__(self):
        return len(self.cards)

    def setup(self):
        for a, b in product(CARD_VALS, SUITS):
            self.cards.append(Card(a, b))
        #self.cards = [Card(a, b) for a in CARD_VALS for b in SUITS]

    def shuffle(self):
        return random.shuffle(self.cards)

    def drawcards(self, nbcards=1):
        result = []
        for i in range(nbcards):
            result.append(self.cards.pop())
        return result
    
    def removecards(self, cards):
        for each in cards:
            self.cards.remove(each)
            
    def addcards(self, cards):
        for each in cards:
            self.cards.append(each)
            
    def addboardcards(self, cards):
        for each in cards:
            self.board.append(each)
    
    def removeboardcards(self, cards):
        for each in cards:
            self.board.remove(each)
        self.addcards(cards)

## The **Player** class is set up with reference to **Deck** and may be passed a **name** or **initial hand**. It can also generate a uuid to add to the PLAYER_DICT
### Player has functions to compare with other players to determine the better one or equality by hand

In [143]:
@total_ordering
class Player:

    def __init__(self, carddeck, name = getpass.getuser(), hand = [], createID = True):
        self.name = name
        self.hand = []
        self.carddeck = carddeck
        if len(hand) > 0:
            for crd in hand:
                self.hand.append(crd)
        if createID:
            self.id = uuid.uuid4()
            PLAYER_DICT[str(self.id)] = self
    
    def __str__(self):
        return "Player: {}\nCards: {}\nPokerhand: {}".format(self.name, self.hand, self.besthand())

    def __repr__(self):
        return "Player({}, {})".format(self.name, self.hand)
        
    def __eq__(self, other):
        #if they both have the same besthand then they are equal
        return self.besthand()[0] == other.besthand()[0]
            
    
    def __gt__(self, other):
        if self.besthand()[1] > other.besthand()[1]:
            return True
        if self.besthand()[1] == other.besthand()[1]:
            return self.besthand()[0] > other.besthand()[0]
        else:
            return False


    def getcards(self, nbcards=1):
        newcards = self.carddeck.drawcards(nbcards)
        for i in newcards:
            self.hand.append(i)
   

    def besthand(self):
        '''
        This function works out what the best possible hand is for a player given their current hand and the board
          player is an instance of Class (Player)
          board is a list with contents only of Class (Card)
        
        It returns a list of [besthand, index], where index is a value from 0 ('High Card') to 8 ('Straight Flush')
        '''
        cardlist = list(self.hand + self.carddeck.board)

        if len(cardlist) < 5:
            return 'Not enough cards for a poker hand'

        #work out each combination of 5 cards and the type of hand they are, then add that set(pokerhandindex, hand) to a list
        handslist = []
        for each in combinations(cardlist, 5):
            handslist.append((pokerhand(each), each))
        
        #creating a default dictionary from this list has the keys = pokerhandindex, values = all poker hands of that index
        handsdict = collections.defaultdict(list)
        for k, v in handslist:
            handsdict[k].append(v)
        
        #the result is the max(hands with the highest pokerhandindex)
        pokerhandindex = max(handsdict.keys())
        result = handsdict[pokerhandindex]
        result = max(map(orderedhand, result))
        return [result, pokerhandindex]
        #return handslist

### The function **pokerhand** returns the type of 5-card poker hand for the list of Cards in **hand** provided

In [144]:
def pokerhand(hand):
    '''
    hand is list of 5 instances of Class (Card)
    returns the name of the poker hand you have
    '''
    if len(hand) != 5:
        return None
    
    #create a collection.Counter dictionary for the card values and the card suits
    #This counts the number of times each card appears in the hand and each suit appears in the hand respectively
    
    card_vals = collections.Counter(v.val for v in hand)
    card_suits = collections.Counter(v.suit for v in hand)
    result = None

    #test for flush. If there are 5 cards of the same suit it's a flush
    if max(card_suits.values()) == 5:
        result = 5

    #test for straight. If the difference of the highest and lowest card values is 4 then it's a straight or test for A-to-5
    #first all the cards must be different values:
    if max(card_vals.values()) == 1:
        #then work out value of highest - lowest index in CARD_VALS
        if max(hand).index - min(hand).index == 4:
            if result == None:
                result = 4
            else:
                result = 8
        elif list(sorted(card_vals.keys())) == ['2','3','4','5','A']:
            if result == None:
                result = 4
            else:
                result = 8
        else:
            if result == None: 
                result = 0

    #test for 4 of a kind
    elif max(card_vals.values()) == 4:
        result = 7

    #test for full house and 3 of a kind
    elif max(card_vals.values()) == 3:
        if min(card_vals.values()) == 2:
            result = 6
        else:
            result = 6

    #test for 1 or 2 pairs
    else:
        #the only possibilities left are 1 or 2 pair
        #count the number of times there is a pair
        if list(card_vals.values()).count(2) == 2:
            result = 2
        else:
            result = 1

    return result


### The function **orderedhand** takes in a list of Cards and returns that list in order, high-to-low from Ace-to-2 but keeps pairs, trips, 4-of-a-kind in order first

In [145]:
def orderedhand(hand):
    carddict={}
    #sorted dictionary by value of cards
    for c in sorted(hand, reverse=True):
        if c.val not in carddict:
            carddict[c.val]=[c]
        else:
            carddict[c.val].append(c)

    #now sort by number of cards into a single list

    #this step creates a list of lists
    result = [carddict[k] for k in sorted(carddict, key=lambda k: len(carddict[k]), reverse=True)]
    #this reduces it to a single list
    result = functools.reduce(operator.iconcat, result, [])        
    return result

### The function **handprobs** runs a number of simulations to produce a probability table of the final poker hand for the **player**

In [146]:
def handprobs(player, nbextracards=0, nbsims=1000):
    '''
    Produces a dictionary of probabilities of the final hand given the starting player and board
    player is any instance of Class (Player)
    deck is an instance of Class (Deck)
    board is a list with contents only of Class (Card)
    Ensure that the deck has had the Cards removed already in Player.hand and board
    nbextracards = number of cards still to draw from the deck to make the player's hand
    nbsims = the number of simulations to run
    '''
    sims = []
    for i in range(nbsims):
        player.carddeck.shuffle()
        newboardcards = player.carddeck.drawcards(nbextracards)
        player.carddeck.addboardcards(newboardcards)
        sims.append(player.besthand()[1])
        player.carddeck.removeboardcards(newboardcards)
    result = {HAND_ORDER[k]:'{:.2f}'.format(v/nbsims*100) 
              for k,v in dict(collections.Counter(sims).most_common(9)).items()}
    return result


### The function **winninghand** returns a **list** of all players with the same best hand being compared

In [147]:
def winninghand(*players):
    ''' 
    *players are as many instances of Class (Player) to compare for the winning hand.
    This function first works out the best hand of all the players using max(players), 
    Then returns a list of all players with the same hand as the result of max(players)
    '''
    mx = max(players)
    result = list(plyr for plyr in players if plyr == mx)
    return result

### The function **comparehandprobs** runs a number of simulations to produce a probability of any one of ***players** winning the hand

In [166]:
def comparehandprobs(nbextracards, nbsims, *players):
    '''
    Given any number of players and an existing deck, the function returns a dictionary of the probability of a player winning
    
    *players must all be an instance of Class (Player)
    Sensible results only if all players have the same Deck
    
    nbextracards = number of cards still to draw from the deck to make the player's hand
    nbsims = the number of simulations to run
    '''
    commondeck = players[0].carddeck
    print('players[0]:', players[0].name)
    print('players[1]:', players[1].name)
    sims = []
    for i in range(nbsims):
        print('\r'+str(i), sep=' ', end='', flush=True)
        commondeck.shuffle()
        newboardcards = commondeck.drawcards(nbextracards)
        commondeck.addboardcards(newboardcards)
        winners = winninghand(*players)
        #         for plyr in winners:
        #             sims.append(str(plyr.id))
        if len(winners) == 1:
            sims.append(winners[0].name)
        else:
#             sims.append(str([plyr.name for plyr in winners]))
            sims.append(functools.reduce(lambda x,y: x+y.name, winners, ''))
        commondeck.removeboardcards(newboardcards)

        result = {k:'{:.2f}'.format(v/nbsims*100)
               for k,v in dict(collections.Counter(sims)).items()}
    return result

# Now some testing

In [149]:
mydeck = Deck()
mydeck.shuffle()
p1 = Player(mydeck, 'Lloyd')
p1.getcards(7)
print(p1.besthand())
print(len(mydeck))

[[Card(5, Clubs), Card(5, Spades), Card(K, Clubs), Card(J, Diamonds), Card(8, Clubs)], 1]
45


In [150]:
mydeck = Deck()

Player1 = Player(
    carddeck = mydeck,
    name = 'Player1',
    hand = [Card('J','Hearts'), Card('K','Hearts'), Card('Q','Hearts'), Card('K','Hearts'), Card('A', 'Spades')]
)


Player2 = Player(
    carddeck = mydeck,
    name = 'Player2',
    hand = [Card('A','Hearts'), Card('K','Hearts'), Card('K','Hearts'), Card('J','Hearts'), Card('10','Clubs')]
)

print('Player 1 == Player2:\t',Player1 == Player2)
print('Player 1 > Player2:\t',Player1 > Player2)
print('pokerhand(Player1.hand):\t', pokerhand(Player1.hand))
print('Player1.besthand():\t', Player1.besthand()[0], HAND_ORDER[Player1.besthand()[1]])
print('orderedhand(Player1):\t', orderedhand(Player1.hand))

print()
# print('Player2.pokerhand():\t',Player2.pokerhand())
# print('Player1.orderedhand():\t',Player1.orderedhand())
# print('Player2.orderedhand():\t',Player2.orderedhand())

Player 1 == Player2:	 False
Player 1 > Player2:	 True
pokerhand(Player1.hand):	 1
Player1.besthand():	 [Card(K, Hearts), Card(K, Hearts), Card(A, Spades), Card(Q, Hearts), Card(J, Hearts)] One pair
orderedhand(Player1):	 [Card(K, Hearts), Card(K, Hearts), Card(A, Spades), Card(Q, Hearts), Card(J, Hearts)]



In [151]:
#Set up some tests
testdeck = Deck()
TestHands = {
    'high_card':Player(testdeck, hand=[Card('A','H'), Card('K','H'), Card('Q','H'), Card('J','H'), Card('9','D')]),
    'one_pair':Player(testdeck, hand=[Card('K','S'), Card('K','H'), Card('Q','H'), Card('J','H'), Card('9','H')]),
    'two_pair':Player(testdeck, hand=[Card('A','H'), Card('A','C'), Card('Q','H'), Card('Q','S'), Card('9','H')]),
    'trips':Player(testdeck, hand=[Card('J','H'), Card('J','D'), Card('J','C'), Card('Q','H'), Card('9','H')]),
    'straight':Player(testdeck, hand=[Card('A','H'), Card('K','H'), Card('Q','H'), Card('J','H'), Card('10','S')]),
    'low_straight':Player(testdeck, hand=[Card('A','H'), Card('2','H'), Card('3','H'), Card('4','H'), Card('5','H')]),
    'flush':Player(testdeck, hand=[Card('A','H'), Card('K','H'), Card('Q','H'), Card('J','H'), Card('9','H')]),
    'full_house':Player(testdeck, hand=[Card('J','H'), Card('J','D'), Card('J','C'), Card('Q','H'), Card('Q','S')]),
    'four_kind':Player(testdeck, hand=[Card('J','H'), Card('J','D'), Card('J','C'), Card('J','S'), Card('9','H')]),
    'straight_flush':Player(testdeck, hand=[Card('A','H'), Card('K','H'), Card('Q','H'), Card('J','H'), Card('10','H')])
    }

In [152]:
maxhand=max(TestHands.values())
minhand=min(TestHands.values())
print('Max hand in TestHands is:\n\t', maxhand.besthand(), '\n\t', maxhand.hand)
print('Min hand in TestHands is:\n\t', minhand.besthand(), '\n\t', minhand.hand)

Max hand in TestHands is:
	 [[Card(A, H), Card(K, H), Card(Q, H), Card(J, H), Card(10, H)], 8] 
	 [Card(A, H), Card(K, H), Card(Q, H), Card(J, H), Card(10, H)]
Min hand in TestHands is:
	 [[Card(A, H), Card(K, H), Card(Q, H), Card(J, H), Card(9, D)], 0] 
	 [Card(A, H), Card(K, H), Card(Q, H), Card(J, H), Card(9, D)]


In [153]:
print(TestHands['full_house'])

repr(TestHands['full_house'])

Player: Lloyd
Cards: [Card(J, H), Card(J, D), Card(J, C), Card(Q, H), Card(Q, S)]
Pokerhand: [[Card(J, H), Card(J, D), Card(J, C), Card(Q, H), Card(Q, S)], 6]


'Player(Lloyd, [Card(J, H), Card(J, D), Card(J, C), Card(Q, H), Card(Q, S)])'

In [154]:
mydeck = Deck()
myhand = [Card('K','Hearts'), Card('K','Spades')]
board = []
mydeck.removecards(myhand)
mydeck.removecards(board)
myplayer = Player(carddeck=mydeck, hand=myhand)

starttime = time.time()
print(handprobs(myplayer, 5, 10000))
endtime = time.time()
print(endtime-starttime)

{'Two pair': '39.82', 'One pair': '36.10', 'Full house': '20.16', 'Flush': '2.00', 'Straight': '1.13', 'Four of a kind': '0.78', 'Straight flush': '0.01'}
4.7291224002838135


In [155]:
winninghand(TestHands['flush'], TestHands['full_house'])[0].besthand()

[[Card(J, H), Card(J, D), Card(J, C), Card(Q, H), Card(Q, S)], 6]

In [171]:
mydeck = Deck()
boardcards = [Card('10', 'Diamonds'), Card('K', 'Clubs'), Card('A', 'Diamonds')]
# boardcards = []
p1hand = [Card('J', 'Diamonds'), Card('3', 'Diamonds')]
p2hand = [Card('A', 'Spades'), Card('4', 'Diamonds')]
mydeck.removecards(p1hand)
mydeck.removecards(p2hand)

p1 = Player(name = 'p1', carddeck = mydeck, hand = p1hand)
p2 = Player(name = 'p2', carddeck = mydeck, hand = p2hand)
mydeck.removecards(boardcards)
mydeck.addboardcards(boardcards)

comparehandprobs(5, 1000, p1, p2)


players[0]: p1
players[1]: p2
999

{'p2': '48.20', 'p1p2': '8.50', 'p1': '43.30'}